<a href="https://colab.research.google.com/github/joyfulspace/ADP/blob/master/10_%EB%82%98%EC%9D%B4%EB%B8%8C%EB%B2%A0%EC%9D%B4%EC%A6%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 나이브 베이즈
* A와 B가 일어날 확률
  - 두 사건이 독립이라면: $P(A)XP(B)$
  - A가 일어났다는 조건 하에서 B가 일어날 확률 $P(B|A)$
  - A와 B가 연달아서 일어날 확률: $P(A)*P(B|A)=P(A\cap B)$ (A가 사전, B가 사후)
  - B가 일어나고 A가 일어날 확률: $P(B)*P(A|B)=P(A\cap B)$
  - 따라서 $P(A)*P(B|A)=P(B)*P(A|B)$ : 베이즈 정리
* 나이브 베이즈: A가 일어나고 B가 일어날 확률과 C가 일어나고 B가 일어날 확률은 다른데 이 원리를 이용
  - 긍정, 부정을 예측하거나 주제 분류, 텍스트 마이닝에서 많이 사용
  - ex. 스팸메일 찾기
    - $P(스팸메일|'광고')=P('광고'|스팸메일)*P(스팸메일)/P('광고')$
    - 광고 단어가 있을 때 스팸메일일 확률 = 스팸메일인데 광고 단어가 있을 확률 * 스팸메일일 확률 / '광고' 단어가 있을 확률
   - 여러개의 단어가 나타났을 때 스팸일지 정상일지?
   - 긴 텍스트라도 쪼개서 확률 계산 가능

In [1]:
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
!git clone https://github.com/GyeeunJeong/ADP.git

Cloning into 'ADP'...
remote: Enumerating objects: 2488, done.
remote: Counting objects: 100% (2181/2181), done.
remote: Compressing objects: 100% (2175/2175), done.
remote: Total 2488 (delta 23), reused 2135 (delta 4), pack-reused 307
Receiving objects: 100% (2488/2488), 55.99 MiB | 24.73 MiB/s, done.
Resolving deltas: 100% (145/145), done.


In [3]:
import chardet

# 영화리뷰데이터 # 파일 없음
with open('ADP/data/imdb_master.csv', 'rb') as f:
  result = chardet.detect(f.read()) # 인코딩 자동으로 감지

train = pd.read_csv('ADP/data/imdb_master.csv', encoding=result['encoding'])
train.head()

FileNotFoundError: ignored

In [ ]:
train = train.drop(train.columns[[0, 1, 4]], axis=1)

In [ ]:
train.label = [(l!='neg')*1 for l in train.label]

In [ ]:
train.head()

In [ ]:
d = train
d.columns = ['user_review', 'positive']
d.head()

NameError: ignored

In [ ]:
# train셋, test셋 나눔
split = 0.7
d_train = d[:int(split*len(d))]
d_test = d[int((1-split)*len(d)):]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
features = vectorizer.fit_transform(d_train.user_review)

In [ ]:
i = 30000 # 3만건에서 100번 째 칸까지 확인하기
j = 100
words = vectorizer.get_feature_names()[i:i+30]
pd.DataFrame(features[j:j+10, i:i+30].todense(), columns=words)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model1 = MultinomialNB()
model1.fit(features, d_train.positive)

pred1 = model1.predict_proba(vectorizer.transform(d_test.user_review))
pred1, pred1.shape

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, roc_curve
from matplotlib.pyplot import plot, xlabel, ylabel, annotate
def performance(y_true, pred, color='g', ann=True):
  acc = accuracy_score(y_true, pred[:,1]>0.5)
  auc = roc_auc_score(y_true, pred[:,1])
  fpr, tpr, thr = roc_curve(y_true, pred[:,1])
  plot(fpr, tpr, color, linewidth='3')
  xlabel('False positive rate')
  ylabel('True positive rate')
  if ann:
    annotate('Acc: %0.2f' % acc, (0.1,0.8), size=14)
    annotate('AUC: %0.2f' % acc, (0.1,0.7), size=14)

In [ ]:
performance(d_test.positive, pred1)